<a href="https://colab.research.google.com/github/palash04/Artificial-Intelligence/blob/master/NN_Series/_Pytorch_CNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
# createt simple CNN
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN,self).__init__()
    # for same convolution : nout = [[(nin + 2p - k) / s] + 1 ] ,
    # where, nin = number of input features
    # nout = number of output features
    # k = convolution kernel size
    # p = convolution padding size
    # s = convolution stride size
    # here, in our case, nin = 28, p = 1, s = 1, k = 3 => nout = 28, which is same as out input feature.. i.e we dont want to reduce the image
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8,kernel_size=(3,3), stride=(1,1), padding=(1,1))
    # for pooling use the same above formula
    # for k = 2, s = 2, we get (14 x 14)
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))   # note here p = 0, by default
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16,kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)    # 16: out_channels, 7 x 7: we did pooling two times (28/2 = 14, then again 14/2 = 7). Hence, 7 x 7

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x

In [3]:
# testing the network's architecture
model = CNN()
x = torch.randn(64, 1, 28, 28)
print (x.shape)

torch.Size([64, 1, 28, 28])


In [4]:
print (model(x).shape)    # 64 x 10 is what we expect to be our final shape coming out of the network

torch.Size([64, 10])


In [5]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
# Hyperparameters
in_channel = 1
num_classes = 10
lr = 0.001
batch_size = 64
epochs = 5

In [8]:
# Load data
train_dataset = datasets.MNIST(root='datatset/', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='datatset/', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Extracting datatset/MNIST/raw/train-images-idx3-ubyte.gz to datatset/MNIST/raw


Extracting datatset/MNIST/raw/train-labels-idx1-ubyte.gz to datatset/MNIST/raw


Extracting datatset/MNIST/raw/t10k-images-idx3-ubyte.gz to datatset/MNIST/raw




Extracting datatset/MNIST/raw/t10k-labels-idx1-ubyte.gz to datatset/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [9]:
# Initialize the Network
model = CNN().to(device)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [15]:
# Train the Network
for epoch in range(1,epochs+1):
  training_loss = 0.0
  for batch_idx, (data,targets) in enumerate(train_loader):
    # get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # forward pass
    output = model(data)

    # calculate loss
    loss = criterion(output, targets)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    training_loss += loss.item()
  
  print (f'Epoch {epoch}/{epochs} \t Trainging Loss: {training_loss/len(train_loader):.6f}')


Epoch 1/5 	 Trainging Loss: 0.054225
Epoch 2/5 	 Trainging Loss: 0.048168
Epoch 3/5 	 Trainging Loss: 0.043423
Epoch 4/5 	 Trainging Loss: 0.039583
Epoch 5/5 	 Trainging Loss: 0.035784


In [16]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print ('Checking for training data')
  else:
    print ('Checking for testing data')

  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)

      output = model(x)
      _, preds = output.max(1)
      num_correct += (preds == y).sum()
      num_samples += preds.size(0)

    print (f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples) * 100 :.6f}')
  model.train()
  

In [17]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking for training data
Got 59468/60000 with accuracy 99.113333
Checking for testing data
Got 9871/10000 with accuracy 98.710000
